In [1]:
import pandas as pd
import numpy as np
import re
import emoji
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
import time
import datetime

nltk.download('words')
words = set(nltk.corpus.words.words())
stop_words = set(stopwords.words('english'))



[nltk_data] Downloading package words to
[nltk_data]     /Users/mosheborochov/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
df = pd.read_json('../data/labeled_datasets/united-2020-12-05-2020-12-15.json')
print(df['text'])
def stemming(words):
    ps=PorterStemmer()
    return [ps.stem(word) for word in words]

def remove_stop_words(words):
    return [c.lower() for c in words if c not in stop_words]

def lemmatizing(words):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]


def cleaner(tweet):
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet).split())
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    return tweet

def final_text(words):
     return ' '.join(words)

#df['text'] = df['text'].map(lambda x: cleaner(x))
#df['text'] = df['text'].apply(lambda x:word_tokenize(x))
#df['text'] = df['text'].apply(lambda x:remove_stop_words(x))
#df['text'] = df['text'].apply(lambda x: lemmatizing(x))
#df['text'] = df['text'].apply(lambda x: stemming(x))
#df['text'] = df['text'].apply(lambda x:final_text(x))

0        RT @DavidCornDC: Hey @United, a family member ...
1        RT @DavidCornDC: Hey @United, a family member ...
2        RT @DavidCornDC: Worse, @United, your flight a...
3        RT @DavidCornDC: Worse, @United, your flight a...
4        RT @DavidCornDC: Hey @United, a family member ...
                               ...                        
34638    RT @aviacionhr_info: Boeing 737-700 de @united...
34639    @CooperWoods04 @BBOZZY33 @united Just saw this...
34640    RT @aviacionhr_info: Boeing 737-700 de @united...
34641    RT @aviacionhr_info: Boeing 737-700 de @united...
34642    @Obsidian_Knee @TomiLahren @united Lol okay De...
Name: text, Length: 34643, dtype: object


In [3]:
df.head()

,id,text,created_at,hashtags,mentions,in_reply_to_user_id,user_id,retweet_count,quote_count,reply_count,like_count,replied_to,retweeted,quoted,is_anomaly
0,1336444683875016704,"RT @DavidCornDC: Hey @United, a family member ...",2020-12-08 22:56:57,[],"[DavidCornDC, united]",NaN,779922519285301248,3151,0,0,0,NaN,1.336362e+18,NaN,0
1,1336444690640465920,"RT @DavidCornDC: Hey @United, a family member ...",2020-12-08 22:56:58,[],"[DavidCornDC, united]",NaN,161003217,3151,0,0,0,NaN,1.336362e+18,NaN,0
2,1336444721556668416,"RT @DavidCornDC: Worse, @United, your flight a...",2020-12-08 22:57:06,[],"[DavidCornDC, united]",NaN,161003217,2056,0,0,0,NaN,1.336362e+18,NaN,0
3,1336444729723064320,"RT @DavidCornDC: Worse, @United, your flight a...",2020-12-08 22:57:08,[],"[DavidCornDC, united]",NaN,794448786,2056,0,0,0,NaN,1.336362e+18,NaN,0
4,1336444732554113024,"RT @DavidCornDC: Hey @United, a family member ...",2020-12-08 22:57:08,[],"[DavidCornDC, united]",NaN,1165791085626396672,3151,0,0,0,NaN,1.336362e+18,NaN,0


In [4]:
#df = df.explode('hashtags').explode('mentions').explode('text')
#df = df.explode('hashtags').explode('mentions')
df = df.explode('hashtags')
continuous_index = []
#symbolic_index = ['hashtags', 'user_id', 'mentions', 'text']
#symbolic_index = ['hashtags', 'mentions']
symbolic_index = ['hashtags']

df_continuous = df.loc[:, continuous_index]
df_symbolic = df.loc[:, symbolic_index]
df_label = df.loc[:, ['is_anomaly']]

In [5]:
df_symbolic.loc[:,'hashtags'].unique()
#df_symbolic.loc[:,'user_id'].unique()
#df_symbolic.loc[:,'mentions'].unique()
#df_symbolic.loc[:,'text'].unique()


hashtags_dict = {}
for i, entry in enumerate(df_symbolic.loc[:,'hashtags'].unique()):
    hashtags_dict[entry] = i
    
'''user_id_dict = {}
for i, entry in enumerate(df_symbolic.loc[:,'user_id'].unique()):
    user_id_dict[entry] = i'''

'''mentions_dict = {}
for i, entry in enumerate(df_symbolic.loc[:,'mentions'].unique()):
    mentions_dict[entry] = i+1
    
text_dict = {}
for i, entry in enumerate(df_symbolic.loc[:,'text'].unique()):
    text_dict[entry] = i'''

"mentions_dict = {}\nfor i, entry in enumerate(df_symbolic.loc[:,'mentions'].unique()):\n    mentions_dict[entry] = i+1\n    \ntext_dict = {}\nfor i, entry in enumerate(df_symbolic.loc[:,'text'].unique()):\n    text_dict[entry] = i"

In [18]:
#df.groupby(df['created_at'].dt.floor('30min')).mean()
bins = [0, 6, 12, 18, 24]
df['Time Bin'] = pd.cut(df['created_at'].dt.hour, bins, right=False)
df['created_at'] = pd.to_datetime(df['created_at']).dt.floor('30T')

In [19]:
df

,id,text,created_at,hashtags,mentions,in_reply_to_user_id,user_id,retweet_count,quote_count,reply_count,like_count,replied_to,retweeted,quoted,is_anomaly,Time Bin,B
0,1336444683875016704,"RT @DavidCornDC: Hey @United, a family member ...",2020-12-08 22:30:00,NaN,"[DavidCornDC, united]",NaN,779922519285301248,3151,0,0,0,NaN,1.336362e+18,NaN,0,"[18, 24)",2020-12-08 22:30:00
1,1336444690640465920,"RT @DavidCornDC: Hey @United, a family member ...",2020-12-08 22:30:00,NaN,"[DavidCornDC, united]",NaN,161003217,3151,0,0,0,NaN,1.336362e+18,NaN,0,"[18, 24)",2020-12-08 22:30:00
2,1336444721556668416,"RT @DavidCornDC: Worse, @United, your flight a...",2020-12-08 22:30:00,NaN,"[DavidCornDC, united]",NaN,161003217,2056,0,0,0,NaN,1.336362e+18,NaN,0,"[18, 24)",2020-12-08 22:30:00
3,1336444729723064320,"RT @DavidCornDC: Worse, @United, your flight a...",2020-12-08 22:30:00,NaN,"[DavidCornDC, united]",NaN,794448786,2056,0,0,0,NaN,1.336362e+18,NaN,0,"[18, 24)",2020-12-08 22:30:00
4,1336444732554113024,"RT @DavidCornDC: Hey @United, a family member ...",2020-12-08 22:30:00,NaN,"[DavidCornDC, united]",NaN,1165791085626396672,3151,0,0,0,NaN,1.336362e+18,NaN,0,"[18, 24)",2020-12-08 22:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34638,1338996404480577536,RT @aviacionhr_info: Boeing 737-700 de @united...,2020-12-15 23:30:00,NaN,"[aviacionhr_info, united, F1Caleb]",NaN,1191728617623097344,12,0,0,0,NaN,1.338945e+18,NaN,0,"[18, 24)",2020-12-15 23:30:00
34639,1338996604238487552,@CooperWoods04 @BBOZZY33 @united Just saw this...,2020-12-15 23:30:00,NaN,"[CooperWoods04, BBOZZY33, united]",1.168008e+18,2468367223,0,0,1,2,1.337825e+18,NaN,NaN,0,"[18, 24)",2020-12-15 23:30:00
34640,1338996618356535296,RT @aviacionhr_info: Boeing 737-700 de @united...,2020-12-15 23:30:00,NaN,"[aviacionhr_info, united, F1Caleb]",NaN,1206188979932016640,12,0,0,0,NaN,1.338945e+18,NaN,0,"[18, 24)",2020-12-15 23:30:00
34641,1338996796312449024,RT @aviacionhr_info: Boeing 737-700 de @united...,2020-12-15 23:30:00,NaN,"[aviacionhr_info, united, F1Caleb]",NaN,1333872907517382656,12,0,0,0,NaN,1.338945e+18,NaN,0,"[18, 24)",2020-12-15 23:30:00


In [25]:
df_symbolic.loc[:,'hashtags'] =  df_symbolic.loc[:,'hashtags'].map(hashtags_dict)
#df_symbolic.loc[:,'user_id'] =  df_symbolic.loc[:,'user_id'].map(user_id_dict)
#df_symbolic.loc[:,'mentions'] =  df_symbolic.loc[:,'mentions'].map(mentions_dict)
#df_symbolic.loc[:,'text'] =  df_symbolic.loc[:,'text'].map(text_dict)
#df_symbolic = df_symbolic.fillna(0)
df_symbolic

,hashtags
0,0
1,0
2,0
3,0
4,0
...,...
34638,0
34639,0
34640,0
34641,0


In [26]:
df_continuous.to_csv("CentralParkNYCnumeric.txt", index=False, header=False)
df_symbolic.to_csv("CentralParkNYCcateg.txt", index=False, header=False)
df_label.to_csv("CentralParkNYC_label.txt", index=False, header=False)
def create_unix(x):
    return int(time.mktime((x).timetuple()))
df.loc[:,'created_at'].map(create_unix).to_csv("CentralParkNYCtime.txt", index=False, header=False)
df['id'].to_csv("CentralParkNYC_tweet_id.txt", index=False)

In [43]:
df['id'].to_csv("CentralParkNYC_tweet_id.txt", index=False)

In [27]:
df_symbolic.shape[0]

36171

In [28]:
text_file = open("CentralParkNYCnumeric.txt", "w")
df_symbolic.shape[0]
n = text_file.write('\n'*df_symbolic.shape[0])
text_file.close()

In [29]:
#df_label.groupby(['is_anomaly']).agg(['min', 'max', 'count', 'nunique'])
#df_label.groupby(['is_anomaly']).agg(['min', 'max', 'count', 'nunique'])
df['is_anomaly'].value_counts()

0    21256
1    14915
Name: is_anomaly, dtype: int64

In [30]:
df.shape

(36171, 15)

In [31]:
df_symbolic

,hashtags
0,0
1,0
2,0
3,0
4,0
...,...
34638,0
34639,0
34640,0
34641,0


In [32]:
df_symbolic.iloc[1]

hashtags    0
Name: 1, dtype: int64

In [131]:
score = pd.read_csv('score.txt', header=None)

In [33]:
df_symbolic

,hashtags
0,0
1,0
2,0
3,0
4,0
...,...
34638,0
34639,0
34640,0
34641,0


In [ ]:
df['id'].to_csv("CentralParkNYC_label.txt", index=False, header=False)




In [44]:
df

,id,text,created_at,hashtags,mentions,in_reply_to_user_id,user_id,retweet_count,quote_count,reply_count,like_count,replied_to,retweeted,quoted,is_anomaly
0,1336444683875016704,"RT @DavidCornDC: Hey @United, a family member ...",2020-12-08 22:56:57,NaN,"[DavidCornDC, united]",NaN,779922519285301248,3151,0,0,0,NaN,1.336362e+18,NaN,0
1,1336444690640465920,"RT @DavidCornDC: Hey @United, a family member ...",2020-12-08 22:56:58,NaN,"[DavidCornDC, united]",NaN,161003217,3151,0,0,0,NaN,1.336362e+18,NaN,0
2,1336444721556668416,"RT @DavidCornDC: Worse, @United, your flight a...",2020-12-08 22:57:06,NaN,"[DavidCornDC, united]",NaN,161003217,2056,0,0,0,NaN,1.336362e+18,NaN,0
3,1336444729723064320,"RT @DavidCornDC: Worse, @United, your flight a...",2020-12-08 22:57:08,NaN,"[DavidCornDC, united]",NaN,794448786,2056,0,0,0,NaN,1.336362e+18,NaN,0
4,1336444732554113024,"RT @DavidCornDC: Hey @United, a family member ...",2020-12-08 22:57:08,NaN,"[DavidCornDC, united]",NaN,1165791085626396672,3151,0,0,0,NaN,1.336362e+18,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34638,1338996404480577536,RT @aviacionhr_info: Boeing 737-700 de @united...,2020-12-15 23:56:34,NaN,"[aviacionhr_info, united, F1Caleb]",NaN,1191728617623097344,12,0,0,0,NaN,1.338945e+18,NaN,0
34639,1338996604238487552,@CooperWoods04 @BBOZZY33 @united Just saw this...,2020-12-15 23:57:22,NaN,"[CooperWoods04, BBOZZY33, united]",1.168008e+18,2468367223,0,0,1,2,1.337825e+18,NaN,NaN,0
34640,1338996618356535296,RT @aviacionhr_info: Boeing 737-700 de @united...,2020-12-15 23:57:25,NaN,"[aviacionhr_info, united, F1Caleb]",NaN,1206188979932016640,12,0,0,0,NaN,1.338945e+18,NaN,0
34641,1338996796312449024,RT @aviacionhr_info: Boeing 737-700 de @united...,2020-12-15 23:58:08,NaN,"[aviacionhr_info, united, F1Caleb]",NaN,1333872907517382656,12,0,0,0,NaN,1.338945e+18,NaN,0


In [49]:
df['id'].duplicated().astype(int)

0        0
1        0
2        0
3        0
4        0
        ..
34638    0
34639    0
34640    0
34641    0
34642    0
Name: id, Length: 36171, dtype: int64

In [47]:
df[~df['hashtags'].isnull()]



,id,text,created_at,hashtags,mentions,in_reply_to_user_id,user_id,retweet_count,quote_count,reply_count,like_count,replied_to,retweeted,quoted,is_anomaly
34,1336445155973349376,"In case you’re traveling @united anytime soon,...",2020-12-08 22:58:49,Fixit,[united],NaN,887366707,0,0,0,0,NaN,NaN,1.336362e+18,0
106,1336446662491910144,First one on delivery.. \n\n@Boeing 737 MAX de...,2020-12-08 23:04:48,aviation,"[Boeing, united]",NaN,1268941614644371456,2,0,0,12,NaN,NaN,NaN,0
106,1336446662491910144,First one on delivery.. \n\n@Boeing 737 MAX de...,2020-12-08 23:04:48,AvGeek,"[Boeing, united]",NaN,1268941614644371456,2,0,0,12,NaN,NaN,NaN,0
174,1336448178636386304,"The first #737MAX Delivery in 21 months, #N275...",2020-12-08 23:10:50,737MAX,"[united, flightradar24]",NaN,1301976096947212288,0,0,0,1,NaN,NaN,NaN,0
174,1336448178636386304,"The first #737MAX Delivery in 21 months, #N275...",2020-12-08 23:10:50,N27519,"[united, flightradar24]",NaN,1301976096947212288,0,0,0,1,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34608,1338990361176367104,@seana30914060 @SydneyLWatson @elizfulop @Ryan...,2020-12-15 23:32:33,Plandemic,"[seana30914060, SydneyLWatson, elizfulop, Ryan...",1.154944e+18,1004753288,0,0,1,1,1.338610e+18,NaN,NaN,0
34609,1338990518710231040,.@United are Nazis 🤦🏼‍♂️ Never flying #UnitedA...,2020-12-15 23:33:11,UnitedAirlines,[united],NaN,1315025645664112640,0,0,0,0,NaN,NaN,NaN,0
34622,1338992945383542784,@TomiLahren @united Love that you cry for canc...,2020-12-15 23:42:50,thebiggesthypocrite,"[TomiLahren, united]",4.686470e+08,1248045786841133056,0,0,0,0,1.338239e+18,NaN,NaN,0
34623,1338993088379953152,@united just want to say ty for kicking the ki...,2020-12-15 23:43:24,mask,[united],2.609076e+08,38445676,0,0,0,1,NaN,NaN,NaN,0


In [50]:
df.loc[:,'id']

0        1336444683875016704
1        1336444690640465920
2        1336444721556668416
3        1336444729723064320
4        1336444732554113024
                ...         
34638    1338996404480577536
34639    1338996604238487552
34640    1338996618356535296
34641    1338996796312449024
34642    1338996951048548352
Name: id, Length: 36171, dtype: int64

In [ ]:
"united_tweet_id.txt"